# Lab 5

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 

Rachel's notes; free to ignore

Rules from: https://bicyclecards.com/how-to-play/blackjack/

Blackjack
- as close to 21 without going over 21
- Betting
    - place bets before deal begins
    - minimum and maximum limits (GR: $2-$500)
    - will ignore splitting pairs (player will not get option to split pairs)
    - will ignore insurance and doubling down
    - if dealer goes over 21, each player that has stood is paid their bet
    - if dealer at 21 or less, pays the bet of any player having a higher total and collects from those with a lower total
        - if dealer and player have equal total, no pay/collection

Game
- to start each player + dealer is given one card faced up, then a second card face up to each player, but dealer gets their second card face down
- stand/hit
    - if bust (over 21), dealer collects the bet wagered
    - Ace can either count as 1 or 11 depending on the player's hand
- Dealer's play
    - once all players have been served by the dealer, the dealer's face down card is turned up
    - Rule: hit when <=16
- drawing plastic card results in shuffling

In [1]:
import numpy as np

In [17]:
#function to mute/allow print commands
def print_message(message,verbose=True):
    if verbose:
        print(message)

In [16]:
import numpy as np
import random

class cards():
    def __init__(self,suit,value):
        self.__suit=suit
        self.__value=value
    
    #accessors
    def suit(self):
        return self.__suit
    
    def value(self):
        return self.__value
    
    def show_card(self):
        print(self.__suit, self.__value)

    def __str__(self):
        return str((self.__suit,self.__value))
    
    __repr__=__str__

class game_deck():
    def __init__(self,n): #where n is the number of decks
        self.__n=n
        self.__deck=[] #list to represent game deck
        self.plastic=cards('plastic','plastic') #plastic card to be inserted into game deck

        #making game deck
        suits=['Clubs','Diamonds','Hearts','Spades']
        face_cards=['Jack','Queen','King','Ace']

        #add n number of decks to the deck list
        for l in range(self.__n):
            for i in range(4): #loop so that cards are added for each suit
                #adding numbered cards
                for j in range(2,11): self.__deck.append(cards(suits[i],j)) 
                #adding face cards
                for k in range(4): self.__deck.append(cards(suits[i],face_cards[k]))

        #adding plastic card
        self.__deck.append(self.plastic)

    def __getitem__(self,key):
        return self.__deck[key]
    
    #method to shuffle deck
    def shuffledeck(self):
        self.__deck=random.sample(self.__deck,len(self.__deck)) #take a random sample of full length of deck, aka shuffling all cards in deck
        return self.__deck
    
    def draw(self):
        #top card is first card in list
        top_card=self.__deck[0]
        #shuffle if card is the plastic card
        if top_card==self.plastic:
            self.__deck=self.shuffledeck() #if plastic card is drawn, shuffle deck
            return self.draw() #draw new top card
        else:
            return top_card

    #removal of card
    def remove(self,card):
        return self.__deck.remove(card)

In [3]:
#testing card class
c=cards('Spades',10)
print(c.suit(),c.value())
c.show_card()

Spades 10
Spades 10


In [3]:
#testing deck class
d=game_deck(2)

#checking that first card is what is drawn
print(d[0])
print(d.draw())
print() #for visual space

#checking shuffle
shuf=d.shuffledeck()
shuf[0]

('Clubs', 2)
('Clubs', 2)



('Diamonds', 3)

In [18]:
#Classes to represent players (human, dealer, set strategy)

class Player():
    def __init__(self,name,hand=[],chips=1000,total=0,bet=0,known=[],hidden=[]):
        self.__name=name #name used to identify in print outputs
        self.__hand=hand #cards in player's hand
        self.__chips=chips #arbitrary amount that I decided players will start with
        self.__total=total
        self.__bet=bet
        self.__known=known
        self.__hidden=hidden
    
    #method for getting sum of values of player's hand
    def hand_total(self):
        #iterating through all cards in hand
        for card in self.__hand:
            #adding value of card if numbered card
            if isinstance(card.value(),int):
                self.__total+=card.value()
            #adding value of card if face card
            elif isinstance(card.value(),str):
                #Jack, Queen, King = 10
                if card.value()!='Ace':
                    self.__total+=10
                #Ace value determination
                else:
                    #if hand is less than 11, than Ace=11 will not bust hand, so value Ace as 11
                    if self.__total<11:
                        self.__total+=11
                    #otherwise Ace should equal 1 to prevent busting hand
                    else:
                        self.__total+=1
        return self.__total
    
    #accessors
    def name(self):
        return self.__name
    
    def player_hand(self):
        return self.__hand
    
    def player_chips(self):
        return self.__chips
    
    def player_bet(self):
        return self.__bet
    
    def known(self):
        return self.__known
    
    def hidden(self):
        return self.__hidden

    #methods to be overwritten by child class (if necessary)

    #method to give each player their initial hand at beginning of round
    def initial_hand(self,deck):
        for i in range(2): #gives players two cards face up
            card=deck.draw() #draw top card from deck
            self.__hand.append(card) #add card to player's hand
            self.__known.append(card) #adding cards drawn into list that represents what cards are known
            deck.remove(card) #remove card from game deck

    #method for what a player is to do when it comes to betting, unless overwritten
    def betting(self):
        player_bet=5 #default betting for Player child classes
        if self.__chips-player_bet<0:
            print(f'{self.__name} is out of chips')
        else:
            return player_bet
    
    #method to be overwritten by child classes
    def player_move(self):
        raise NotImplementedError
    
    #method to determine if a player wins/loses bet
    def pay_or_collect(self,dealer,verbose):
        #collects bet if player busts
        if self.__total>21:
            self.__chips-=self.__bet
        #player payed if (player total>dealer total) or (dealer bust and player<21)
        elif (self.__total>dealer.hand_total()) or (dealer.hand_total()>21 and self.__total<=21):
                self.__chips+=self.__bet
        #note: no chips paid/collected if player total = dealer total
        #print out players' chip status
        print_message(f'{self.__name} chips: {self.__chips}',verbose)

    #method for resetting players between rounds
    def reset(self):
        self.__hand=[] 
        self.__total=0
        self.__bet=0
        #self.__known=[]
        #self.__hidden=[]
    
class Dealer(Player):
    def __init__(self,name,hand=[],chips=1000,total=0,bet=0,known=[],hidden=[]):
        Player.__init__(self,name,hand=[],chips=1000,total=0,bet=0,known=[],hidden=[])

    #overriding parent method
    def initial_hand(self,deck):
        for i in range(2):
            #add card face up for dealer first time around
            if i==0:
                card=deck.draw()
                self.player_hand().append(card)
                self.known().append(card)
                deck.remove(card)
            #add card face down for dealer the second time around
            else:
                card=deck.draw()
                self.player_hand().append(card)
                self.hidden().append(card) #add face down card to hidden list that represents cards drawn that are not known to players
                deck.remove(card)

    #method that determines dealer/dealer-style player's strategy
    def player_move(self,deck):
        #dealer hits on 16
        if self.hand_total()<17:
            new_card=deck.draw()
            self.player_hand().append(new_card)
            self.known().append(new_card)
        else:
            return 'stood' #output used for looping in game function further down

class Human(Player):
    def __init__(self,name,hand=[],chips=1000,total=0,bet=0,known=[],hidden=[]):
        Player.__init__(self,name,hand=[],chips=1000,total=0,bet=0,known=[],hidden=[])

    #overriding method, ask for user to input amount they want to bet
    def betting(self):
        hbet=int(input('Bet amount (min: 1, max: 500): ')) #arbitrary min/max chosen
        while self.player_chips()-hbet<0 or hbet>500 or hbet<1:
            hbet=int(input('Bet amount (min: 1, max: 500): ')) #if user does not follow min/max; keep asking for input
        return hbet

    #overriding method, ask user for input to make move hit/stand
    def player_move(self,deck):
        #ask to hit or stand to user
        hmove=input('Hit or stand? ')
        hmove.lower() #make input uniform (all lowercase) for code to understand

        #if user says 'hit' draw card and add to player's hand and to known list of cards
        if hmove=='hit':
            new_card=deck.draw()
            self.player_hand().append(new_card)
            self.known().append(new_card)
        else:
            return 'stood'

class Q6Player(Player):
    def __init__(self,name,hand=[],chips=1000,total=0,bet=0,known=[],hidden=[]):
        Player.__init__(self,name,hand=[],chips=1000,total=0,bet=0,known=[],hidden=[])

    #no betting function, will just use default in parent class

    #method that gives numerical value to represent strategy
    def card_counting(self,knowncards):
        #initial start with 0 value
        comp=0

        #iterating through knowncards and summing values
        for card in knowncards: #iterating through all known cards
            if isinstance(card,int):
                #adding
                if card.value()<7 and card.value()>=2:
                    comp+=1 #add one if card less than 7 and greater than or equal to 2
                elif card.value()<10 and card.value()>=7:
                    comp+=0 #no change when cards are less than 10 and greater than or equal to 7
            else:
                comp+=-1 #subtract 1 in all other instances (10,Jack,Queen,King,Ace)
        
        #computated sum returned as output to be used to hit/stand
        return comp
    
    #overriding method and implementing the strategy method above
    def player_move(self,deck):    
        #Question 6 strat; threshold chosen = -3
        if self.card_counting(self.known())<-3: #low number indicates multiple high numerical value cards were played, player less likely to bust
            new_card=deck.draw()
            self.player_hand().append(new_card)
            self.known().append(new_card)
        else:
            return 'stood'


In [19]:
#class to represent all the data held within a game of blackjack
class Blackjack():
    def __init__(self,number_decks,rounds,players):
        self.__number_decks=number_decks
        self.__rounds=rounds
        self.__players=players #players should be a list of Player objects
        self.__dealer=Dealer('The Dealer')
        self.__gamedeck=game_deck(self.__number_decks)

    #accessors
    def number_decks(self):
        return self.__number_decks
    
    def players(self):
        return self.__players

    def rounds(self):
        return self.__rounds
    
    def dealer(self):
        return self.__dealer
    
    def gamedeck(self):
        return self.__gamedeck
    

In [89]:
#testing Player class
player1=Player('player1')
player1.player_hand() #should return empty list


[]

In [90]:
#testing Player class continued; all should reflect initialization numbers
print(player1.hand_total()) 
print(player1.player_bet())
print(player1.player_chips())

0
0
1000


In [93]:
#testing Blackjack class
player2=Player('player2')
b=Blackjack(2,5,[player1,player2])
print(b.gamedeck()[53]) #check that gamedeck is actually two decks combined
print()

#testing rounds accessor; expecting 5
print(b.rounds())
print()

#testing other attribute accessors, expecting list of Player class objects and empty lists
print(b.players())
print(b.known())
print(b.hidden())

('Clubs', 3)

5

[<__main__.Player object at 0x1074fe630>, <__main__.Player object at 0x1074fa3f0>]
[]
[]


In [20]:
#function that wraps all the classes to perform a game of Blackjack
def BJGame(blackjack,verbose=True):
    #check blackjack class
    if not isinstance(blackjack,Blackjack):
        TypeError
    else: #the game initiates
        #start at round 1
        game_round=1
        dk=blackjack.gamedeck() #creating game deck
        df=dk.shuffledeck() #shuffling game deck

        #looping while round count is less than the number of rounds indicated in blackjack class object
        while game_round<=blackjack.rounds():
            #Print out what round
            print_message(f'Round {game_round}',verbose) #print output to show player what round it is

            #set up initial hands for players/dealer (class methods called)
            for player in blackjack.players():
                player.initial_hand(dk)
            blackjack.dealer().initial_hand(dk)

            #betting for round (class method called)
            for player in blackjack.players(): #iterating through all players
                player.__bet=player.betting()
                #initial status to allow for looping
                stand=False
                dstand=False
            
            #Round starts (actual card game)
            #looping to allow player to continue calling hit, until they call stand
            while stand==False:
                for player in blackjack.players():
                    #each player can hit, until they stand, then next player turn
                    print_message(f'{player.name()}: {player.player_hand()}',verbose)
                    mv=player.player_move(dk)
                    if mv=='stood':
                        stand=True #breaks loop for current player
                    else:
                        print_message(f'{player.name()}: {player.player_hand()}',verbose)
                        mv=player.player_move(dk) #loops asking for player's move (hit/stand)
            
            #The Dealer's turn; same as player but for The Dealer of game
            while dstand==False:
                print_message(f'{blackjack.dealer().name()}: {blackjack.dealer().player_hand()}',verbose)
                mv=blackjack.dealer().player_move(dk)
                if mv=='stood':
                    dstand=True
                else:
                    print_message(f'{blackjack.dealer().name()}: {blackjack.dealer().player_hand()}',verbose)
                    mv=blackjack.dealer().player_move(dk)

            #Round finished; bets paid/collect (call class method)
            #iterating through all players
            for player in blackjack.players():
                player.pay_or_collect(blackjack.dealer(),verbose)        

            #round count updated + reset
            game_round+=1
            for player in blackjack.players(): #iterating through all players
                player.reset() #call reset class method
            blackjack.dealer().reset()
            #recollecting cards used in round + shuffle    
            dk=blackjack.gamedeck()
            df=dk.shuffledeck()

        print_message('Blackjack game ended',verbose)
        
        results=[] #empty list to hold players chip count at end of game
        for player in blackjack.players():
            results.append(player.player_chips())
        #printing players' results (chips left at end of game)
        return results



In [7]:
#testing with one human player and two dealer players
p1=Human('p1')
p2=Dealer('p2-dealer')
p3=Dealer('p3-dealer')
players_list=[p1,p2,p3]

game=Blackjack(6,2,players_list)
BJGame(game)

Round 1
p1: [('Diamonds', 'King'), ('Diamonds', 3)]
p1: [('Diamonds', 'King'), ('Diamonds', 3), ('Clubs', 2)]
p2-dealer: [('Clubs', 3), ('Spades', 3)]
p2-dealer: [('Clubs', 3), ('Spades', 3), ('Clubs', 2)]
p3-dealer: [('Spades', 9), ('Hearts', 2)]
p3-dealer: [('Spades', 9), ('Hearts', 2), ('Clubs', 2)]
p1: [('Diamonds', 'King'), ('Diamonds', 3), ('Clubs', 2)]
p2-dealer: [('Clubs', 3), ('Spades', 3), ('Clubs', 2), ('Clubs', 2)]
p3-dealer: [('Spades', 9), ('Hearts', 2), ('Clubs', 2)]
The Dealer: [('Hearts', 8), ('Clubs', 6)]
The Dealer: [('Hearts', 8), ('Clubs', 6), ('Clubs', 2)]
The Dealer: [('Hearts', 8), ('Clubs', 6), ('Clubs', 2)]
p1 chips: 1000
p2-dealer chips: 1000
p3-dealer chips: 1000
Round 2
p1: [('Spades', 6), ('Clubs', 4)]
p1: [('Spades', 6), ('Clubs', 4), ('Hearts', 3)]
p2-dealer: [('Clubs', 5), ('Diamonds', 5)]
p2-dealer: [('Clubs', 5), ('Diamonds', 5), ('Hearts', 3)]
p3-dealer: [('Diamonds', 5), ('Clubs', 9)]
p3-dealer: [('Diamonds', 5), ('Clubs', 9), ('Hearts', 3)]
p1: [('

[1000, 1000, 1000]

In [21]:
#testing scenario player
sp1=Q6Player('Scenario Player')
sp2=Dealer('p2')
sp3=Dealer('p3')
sp4=Dealer('p4')
plist=[sp1,sp2,sp3,sp4]

q6_game=Blackjack(6,50,plist)
BJGame(q6_game,verbose=False)

RecursionError: maximum recursion depth exceeded

In [ ]:
#100 game loop
def game_loop(loops,game_func):
    game_results=[] #list to hold results from each game
    #looping blackjack game
    for i in range(loops):
        game_results+=game_func #adding results to list
    return game_results #returning all results

In [ ]:
#Qustion 8, running loop
results=game_loop(100,BJGame(q6_game))
res=np.array(results) #list of list into array
res=res[:,:-1]

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#iterating through players
for i in range(res.shape[1]):
    plt.hist(res[:,i],label='Player '+str(i),alpha=0.5) #plotting histogram of each players' results over the 100 games on a single plot
plt.legend() #legend so you can tell which player is which histogram
plt.show()

In [ ]:
#calculating mean and standard deviation
m=np.mean(res,axis=0)
s=np.std(res,axis=0)
plt.errorbar(range(len(m)),m,s) #graphing the error, variability of each player